# Mathematik für Biologiestudierende II

Sommersemester 2025

22.04.2025

&copy; 2025 Prof. Dr. Rüdiger W. Braun 

In [ ]:
import numpy as np
np.set_printoptions(legacy='1.21')
import pandas as pd
from scipy import stats
import seaborn as sns
sns.set_theme()

# Post-hoc Analyse

* Wenn die ANOVA einen signifikanten Unterschied zwischen den Gruppen gezeigt hat, dann versucht man mit der post-hoc Analyse herauszubekommen, zwischen welchen einzelnen Gruppen signifikante Unterschiede bestehen
* Die post-hoc Analyse muss für multiple Vergleiche korrigiert werden

### Beispiel Zitronen

In [ ]:
df = pd.read_csv("http://reh.math.uni-duesseldorf.de/~braun/bio2425/zitronen.csv")

In [ ]:
df.head()

Die Tabelle (mit erfundenen Daten) zeigt den Vitamin C Gehalt in [mg] pro [kg] von Zitronen aus verschiedenen Ländern

In [ ]:
df.Land.value_counts()

In [ ]:
sns.displot(df, x='Vitamin_C_Gehalt', hue='Land', multiple='stack');

In [ ]:
spanien = df[df.Land=='Spanien'].Vitamin_C_Gehalt
italien = df[df.Land=='Italien'].Vitamin_C_Gehalt
griechenland = df[df.Land=='Griechenland'].Vitamin_C_Gehalt
marokko = df[df.Land=='Marokko'].Vitamin_C_Gehalt
indien = df[df.Land=='Indien'].Vitamin_C_Gehalt

In [ ]:
stats.f_oneway(spanien, italien, griechenland, marokko, indien)

Die Unterschiede zwischen den Vitamin C Gehalten sind signifikant

## Paarvergleiche

* Wir könnten zwischen je zwei Gruppen die Paarvergleiche zu Fuß ausrechnen und Bonferroni-korrigieren
* Dieser Prozess ist aber implementiert

In [ ]:
from statsmodels.sandbox.stats.multicomp import MultiComparison

Achtung:  Hier wird irgendwann der Bestandteil `sandbox` überflüssig

In [ ]:
muc = MultiComparison(df.Vitamin_C_Gehalt, df.Land)

`muc = MultiComparison(daten_liste, gruppen_liste)`

* das erste Element von `daten_liste` gehört zur ersten Gruppe in Gruppenliste
* das zweite Element von `daten_liste` gehört zur zweiten Gruppe in Gruppenliste
* usw.

`muc.allpairtest(test, alpha, method)`

* Paarvergleiche zwischen allen Paaren von Gruppen aus der Gruppenliste, mit der `muc` angelegt wurde
* `test` ist der einzusetzende Test
* `alpha` das Signifikanzniveau (Standardwert ist 0.05)
* `method` die Korrekturmethode für das multiple Testen, für uns relevant:
  * `bonferroni`: Bonferroni
  * `holm`: Bonferroni-Holm

In [ ]:
res = muc.allpairtest(stats.ttest_ind, method='bonferroni')
res[0]

Nur vier der Paarvergleiche sind signifikant, wenn Bonferroni korrigiert wird

Dasselbe mit Bonferroni-Holm

In [ ]:
res = muc.allpairtest(stats.ttest_ind, method='holm')
res[0]

* Wenn Bonferroni-Holm korrigiert wird, dann sind sechs der Paarvergleiche signifikant

* Es hängt von der Fächerkultur ab, ob Bonferroni-Holm akzeptiert wird

## Bonferroni-Holm

* n multiple Vergleiche werden durchgeführt
* Die p-Werte werden der Größe nach geordnet
* der kleinste p-Wert muss signifikant zu $\frac\alpha n$ sein
* der zweitkleinste zu $\frac\alpha{n-1}$
* drittkleinste zu $\frac\alpha{n-2}$
* usw.
* der größte zum Niveau $\alpha$

Bei den Zitronen

* der kleinste *p*-Wert ist der des Vergleichs zwischen Indien und Italien
* er ist unkorrigiert gleich 2.311e-5 
* es gibt 10 Paarvergleiche, also ist der Bonferroni-korrigierte Wert gleich 0.0002311
* die Bonferroni-Holm Korrektur führt zum selben Wert


* der zweitkleinste *p*-Wert ist der des Vergleichs zwischen Griechenland und Indien
* er ist unkorrigiert gleich 2.125e-4
* also mit Bonferroni-Korrektur gleich 0.002125
* nach Bonferroni-Holm ist der gleich 2.125e-4/9 = 1.913

## Ablesung genauerer Werte

* Woher weiss ich die genauen *p*-Werte?
* `res = muc.allpairtest(stats.ttest_ind, method='bonferroni')` ist ein Paar mit zwei Einträgen
* `res[0]` ist die leserfreundlich formatierte Tabelle
* die genauen Werte stehen in `res[1]`

In [ ]:
res[1] 

* Der erste Array ist zweidimensional und enthält die genauen Werte der Statistik und die unkorrigierten *p*-Werte

* wichtig ist der dritte Array, der die korrigierten *p*-Werte in der Reihenfolge enthält in der die Paarvergleiche in der Tabelle aufgeführt sind

In [ ]:
p_werte_korrigiert = res[1][2]
p_werte_korrigiert

In [ ]:
stats.f_oneway(spanien, italien, griechenland, marokko, indien).pvalue

Der *p*-Wert der ANOVA ist kleiner als 5.0E-6.  Daher ist zu diesem Signifikanzniveau nachgewiesen, dass Zitronen aus verschiedenen Ländern unterschiedliche Vitamin C Gehalte haben.  Wir rechnen die post-hoc Analyse für dieses Signifikanzniveau

In [ ]:
res = muc.allpairtest(stats.ttest_ind, alpha=5.0E-6, method='bonferroni')

In [ ]:
res[0]

Keiner der Paarvergleiche ist signifikant.

# Behandlung von NaN

In [ ]:
df = sns.load_dataset('penguins')
df.head()

In [ ]:
df.island.value_counts()

In [ ]:
g1 = df[df.island=='Biscoe'].body_mass_g.dropna()
g2 = df[df.island=='Dream'].body_mass_g.dropna()
g3 = df[df.island=='Torgersen'].body_mass_g.dropna()

In [ ]:
stats.f_oneway(g1, g2, g3)

* `allpairtest` benötigt alle Daten in derselben Tabelle

* Wir müssen also in der Ausgangstabelle alle Zeilen löschen, in denen das Gewicht fehlt

In [ ]:
df_dropped = df[df.body_mass_g.notnull()]
df_dropped

* man möchte testen `df.body_mass_g==np.nan`
* das geht aber nicht, weil `nan` besondere Rechenregeln hat
* stattdessen prüft die Methode `notnull()` darauf, ob ein Element ungleich `NaN` ist
* das Gegenteil von `notnull` ist `isnull`

Im Gegensatz dazu entfernt die folgende Operation alle Zeilen, in denen mindestens ein Wert fehlt

In [ ]:
df2 = df.dropna()
df2

* Das sind 9 Zeilen weniger
* Es gibt also 9 Pinguine, von denen das Gewicht bestimmt werden konnte, mindestens ein anderer Wert aber nicht

In [ ]:
df[df.body_mass_g.notnull() & df.sex.isnull()]

In [ ]:
muc = MultiComparison(df_dropped.body_mass_g, df_dropped.island)

In [ ]:
res = muc.allpairtest(stats.ttest_ind, method='bonferroni')

In [ ]:
res[0]

* Wir haben einen *Störfaktor* (engl. *confounding variable*)
* Das ist eine unbeachtete Größe, die die Zielvariable beeinflusst 

* Das Gewicht hängt von der Art ab
* Nicht auf allen Inseln sind alle Arten im selben Umfang vertreten

In [ ]:
sns.displot(df, x="body_mass_g", col="island", hue="species", multiple='stack');

In [ ]:
muc = MultiComparison(df_dropped.body_mass_g, df_dropped.species)
res = muc.allpairtest(stats.ttest_ind, method='bonferroni')
res[0]

* Adelie- und Zügelpinguine wiegen gleich viel
* Eselspinguine unterscheiden sich im Gewicht
* Eselspinguine gibt es nur auf Biscoe